# OBeLiX - descriptor calculator tutorial
By: Adarsh Kalikadien

The source code for OBeLiX can be found at https://github.com/epics-group/obelix and the documentation is on the same page. 

In this tutorial, we will go through various stages of calculating descriptors. 

In [1]:
import os

import pandas as pd
from obelix.descriptor_calculator import Descriptors

We start simple. We have XYZ files of our structures (metal center + NBD + bidentate ligand). The descriptor calculator uses a function called calculate_morfeus_descriptors() for these descriptors. It is also possible to print the steric map of the buried volume in this function. Try it out! 

In [2]:

# calculate descriptors from xyz file

descriptors = Descriptors(central_atom='Rh',
                          path_to_workflow=os.path.join(os.getcwd(), 'xyz_files'),
                          output_type='xyz')
descriptors.calculate_morfeus_descriptors(geom_type='BD',
                                          solvent=None,
                                          printout=False,
                                          metal_adduct='nbd', # 'pristine'
                                          plot_steric_map=False)
descriptors.descriptor_df.to_csv('xyz_descriptors.csv', index=False)

  0%|          | 0/20 [00:00<?, ?it/s]


Calculating descriptors for:  L1 ...
NBD found at end of xyz file, calculating stuff the easy way


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

  5%|▌         | 1/20 [00:11<03:36, 11.41s/it]


Calculating descriptors for:  L10 ...
NBD found at end of xyz file, calculating stuff the easy way


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

 10%|█         | 2/20 [00:22<03:23, 11.33s/it]

...

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

 90%|█████████ | 18/20 [03:47<00:27, 13.85s/it]


Calculating descriptors for:  L8 ...
NBD found at end of xyz file, calculating stuff the easy way


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

 95%|█████████▌| 19/20 [04:00<00:13, 13.78s/it]


Calculating descriptors for:  L9 ...
NBD found at end of xyz file, calculating stuff the easy way


100%|██████████| 20/20 [04:16<00:00, 12.81s/it]


The descriptors will be written to a csv file called xyz_descriptors.csv. 
Now, we can use this same function to calculate descriptors over a CREST conformer ensemble and boltzmann average them. Note, this implementation always boltzmann averages (using the xTB energy) all descriptors. However, on another branch (https://github.com/EPiCs-group/obelix/tree/confomer_searching_dev_final) of the Github repository we have a different implementation where the descriptors for each conformer individiually are calculated. 

In [ ]:
conformer_descriptors = Descriptors(central_atom='Rh',
                                    path_to_workflow=os.path.join(os.getcwd()),
                                    output_type='crest')
conformer_descriptors.calculate_morfeus_descriptors(geom_type='BD',
                                                    solvent=None,
                                                    printout=False,
                                                    metal_adduct='pristine')
conformer_descriptors.descriptor_df.to_csv('conformer_descriptors.csv', index=False)

You can see in conformer_descriptors.csv that the boltzmann average, standard deviation and variance per descriptor is calculated.

Now, say that we have done DFT calculations on our structures. Now we can go to calculating the DFT descriptors. Examples of DFT log files can be found in the dft_log_files/ folder. 
In this case we use the calculate_dft_descriptors_from_log() function. This function makes use of calculate_morfeus_descriptors() and a DFTExtractor() class with its own functions to extract the descriptors from Gaussian log files. We can also extract the XYZ file of the optimized structure from the log file if needed.

In [3]:
path_to_dft_log_files = os.path.join(os.getcwd(), 'dft_log_files')
dft_descriptors = Descriptors(central_atom='Rh',
                              path_to_workflow=path_to_dft_log_files,
                              output_type='gaussian')
dft_descriptors.calculate_dft_descriptors_from_log(geom_type='BD',
                                                   solvent=None,
                                                   extract_xyz_from_log=True,
                                                   printout=False,
                                                   metal_adduct='nbd',
                                                   plot_steric_map=False)
dft_descriptors.descriptor_df.to_csv('DFT_descriptors.csv', index=False)

  0%|          | 0/20 [00:00<?, ?it/s]


Calculating descriptors for: L1
NBD found at end of xyz file, calculating stuff the easy way


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

  5%|▌         | 1/20 [00:15<04:46, 15.06s/it]


Calculating descriptors for: L10
NBD found at end of xyz file, calculating stuff the easy way


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

 10%|█         | 2/20 [00:33<05:05, 16.97s/it]

...

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

 90%|█████████ | 18/20 [05:42<00:39, 19.94s/it]


Calculating descriptors for: L8
NBD found at end of xyz file, calculating stuff the easy way


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

 95%|█████████▌| 19/20 [05:58<00:18, 18.93s/it]


Calculating descriptors for: L9
NBD found at end of xyz file, calculating stuff the easy way


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

100%|██████████| 20/20 [06:21<00:00, 19.09s/it]


If you check DFT_descriptors.csv, next to the block of NBD and Morfeus descriptors, there is also a block of DFT-based descriptors.

Can you now check how descriptors from the xyz_descriptors.csv have changed with respect to the DFT_descriptors.csv? How would you visualize this? Perform this analysis in the code block below, use of AI assistance is allowed :)

We have calculated descriptors of the metal-ligand complex, but we also want to calculate descriptors on the free ligand without influence from the metal and susbstrate. 
We use the molecular_graph() of obelix, that is used to find the ligand atoms in a complex, to extract the free ligand as an XYZ file. The comment line of this XYZ file contains the atom number of donor 1 and donor 2 in both the complex and the free ligand. This makes it easier for us to match the min/max donor labelling later.

In [ ]:
# extract free ligand from Gaussian log file
# iterate over log files and extract the free ligand as an xyz file
from tqdm import tqdm
from morfeus.io import read_cclib, write_xyz
from obelix.molecular_graph import molecular_graph
import glob

path_to_dft_log_files = os.path.join(os.getcwd(), 'dft_log_files')
complexes_to_calc_descriptors = glob.glob(os.path.join(path_to_dft_log_files, '*.log'))
dictionary_for_properties = {}

for metal_ligand_complex in tqdm(complexes_to_calc_descriptors):
    ########## read the file name, coordinates and elements from a log file
    elements, coordinates = read_cclib(metal_ligand_complex)
    if not len(coordinates[-1]) == 3:
        coordinates = coordinates[-1]
    base_with_extension = os.path.basename(metal_ligand_complex)
    split_base = os.path.splitext(base_with_extension)
    filename = split_base[0]
    ##########
    # extract the free ligand from the complex' log file 
    molecular_graph(elements=elements, coordinates=coordinates, extract_ligand=True, path_to_workflow=path_to_dft_log_files, filename=filename)

After extracting the free ligand as 'free_ligand_L*.xyz' we can use it to do a DFT single-point calculation. 

When DFT single-point calculations are done, the xyz and the log file of the free ligand should be placed in the same location. An example can be found in the free_ligand/ folder. Now we can use the information about the donor atoms from the free ligand to match it with the descriptors of the complex that we calculated. This ensures that we keep the same orientation of our descriptors. Finally, we create a free_ligand_descriptors.csv with the descriptors of the free ligand that match the orientation of the complex.

In [ ]:
# calculate free ligand descriptors
from obelix.free_ligand import FreeLigand
from obelix.tools.utilities import dataframe_from_dictionary

ligand_number_list = ['L9', 'L10']
dictionary_for_properties = {}
complex_descriptors_df = pd.read_csv('DFT_descriptors.csv')

for ligand_number in ligand_number_list:
    path_to_free_ligand_files = os.path.join('free_ligand')
    free_ligand = FreeLigand(path_to_workflow=path_to_free_ligand_files,
                             xyz_filename=f'free_ligand_{ligand_number}.xyz',
                             dft_log_file=f'free_ligand_{ligand_number}_SP.log')
    # read the min/max donor indices from the excel file and match them with free_ligand.complex_xyz_bidentate_1_idx and free_ligand.complex_xyz_bidentate_2_idx
    # if complex_xyz_bidentate_1 is min donor, then free_ligand_xyz_bidentate_1 is min donor for our free ligand class
    # first check if index_donor_min from excel is equal to complex_xyz_bidentate_1_idx or complex_xyz_bidentate_2_idx
    complex_bidentate_min_donor_idx = \
        complex_descriptors_df.loc[complex_descriptors_df['filename_tud'] == ligand_number, 'index_donor_min'].values[0]
    if complex_bidentate_min_donor_idx == free_ligand.complex_xyz_bidentate_1_idx:
        # if this is true, then the free ligand bidentate 1 is the min donor
        free_ligand.min_donor_idx = free_ligand.free_ligand_xyz_bidentate_1_idx
        free_ligand.max_donor_idx = free_ligand.free_ligand_xyz_bidentate_2_idx
    else:
        # otherwise the free ligand bidentate 2 is the min donor
        free_ligand.min_donor_idx = free_ligand.free_ligand_xyz_bidentate_2_idx
        free_ligand.max_donor_idx = free_ligand.free_ligand_xyz_bidentate_1_idx

    free_ligand.initialize_dft_extractor(free_ligand.dft_log_file, None, free_ligand.min_donor_idx,
                                         free_ligand.max_donor_idx, metal_adduct='pristine')
    # free_ligand.assign_min_max_donor_dft()
    properties = free_ligand.calculate_dft_descriptors()
    dictionary_for_properties[ligand_number] = properties
new_descriptor_df = dataframe_from_dictionary(dictionary_for_properties)
# reset the index and name that column to 'Ligand#' for consistency with the complex descriptor df
new_descriptor_df = new_descriptor_df.reset_index().rename(columns={'index': 'Ligand#'})
new_descriptor_df.to_csv('free_ligand_descriptors.csv', index=False)

Now you can merge the DFT_descriptors.csv and free_ligand_descriptors.csv to get a DFT-based representation of your catalyst for ML.